# The One Max Problem

**Genetic Algorithms** involve three main operators:

- **Mutation**: Change a candidate slightly
- **Crossover**: Combine two candidates
- **Selection**: Kill a candidate or let it live based on a fitness function

Repeat over many generations

In [30]:
import random
from deap import base, creator, tools
import ipywidgets as widgets


In [31]:
# Goal: Generate a 75-bit pattern with 45 ones

def create_toolbox(num_bits, indpb, tournsize, target_sum):
    def evaluate(individual):
        # Specifically, maximize this function
        return len(individual) - abs(sum(individual) - target_sum)
    
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("map", futures.map)

    # tools.initRepeat fills a container with a function called n times
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, num_bits)

    # Population is a list of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # The evaluation operator
    toolbox.register("evaluate", evaluate)

    # The crossover operator
    toolbox.register("mate", tools.cxTwoPoint)

    # The mutation operator
    toolbox.register("mutate", tools.mutFlipBit, indpb=indpb)
        
    # The selection operator
    toolbox.register("select", tools.selTournament, tournsize=tournsize)
        
    return toolbox   

In [32]:
def evolve(num_bits=75, indpb=0.05, tournsize=3, target_sum=45, n=500, prob_cross=0.5, prob_mut=0.2, num_generations=60):

    toolbox = create_toolbox(num_bits, indpb, tournsize, target_sum)
    random.seed(7)
    
    population = toolbox.population(n=n)
    
    print("Starting the Evolution Process")
    fitnesses = list(map(toolbox.evaluate, population))

    for ind, fit in zip(population, fitnesses):
        # For each individual in the population...
        ind.fitness.values = (fit,)
    
    print("Evaluated {0} individuals".format(len(population)))
    print()
    
    for g in range(num_generations):
        print("Generation #{0} =============".format(g))
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            # Iterate over the offspring in pairs
            if random.random() < prob_cross:
                toolbox.mate(child1, child2)
                
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < prob_mut:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        invalid_individuals = [i for i in offspring if not i.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_individuals)
        
        for i, f in zip(invalid_individuals, fitnesses):
            i.fitness.values = (f,)
        
        print("Re-evaluated {0} individuals".format(len(invalid_individuals)))
        population[:] = offspring
        fits = [i.fitness.values[0] for i in population]
        
        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(f*f for f in fits)
        std = abs(sum2 / length - mean**2) ** 0.5
        
        print("Min = {0:.2f}, Max = {1:.2f}, Average = {2:.2f}, Stddev = {3:.2f}".format(min(fits), max(fits), mean, std))
        print()
    
    best = tools.selBest(population, 1)[0]
    print("Best Individual:", best)
    print("# of ones:", sum(best))

widgets.interactive(
    evolve,
    num_bits=widgets.IntSlider(value=75, min=8, max=512, continuous_update=False),
    indpb=widgets.FloatSlider(value=0.05, min=0.01, max=1.0, step=0.01, continuous_update=False),
    tournsize=widgets.IntSlider(value=3, min=2, max=8, continuous_update=False),
    target_sum=widgets.IntSlider(value=45, min=0, max=512, continuous_update=False),
    n=widgets.IntSlider(value=500, min=10, max=5000, continuous_update=False),
    prob_cross=widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.1, continuous_update=False),
    prob_mut=widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.1, continuous_update=False),
    num_generations=widgets.IntSlider(value=60, min=2, max=250, continuous_update=False)
)

interactive(children=(IntSlider(value=75, continuous_update=False, description='num_bits', max=512, min=8), Fl…